# Linear Programming Turorial: 
## 2) Parametrized Furniture Factory Problem

## Objective and prerequisites

The goal of this Jupyter Notebook is to describe a parametrized version of the Furniture Factory Problem discussed in the
videos: *Chapter 6, Modeling and Solving Linear Programming Problems* and
*Chapter 7, Sensitivity Analysis of Linear Programming Problems*, of the [LP Tutorial](https://www.gurobi.com/resource/mathematical-programming-tutorial-linear-programming/).
The Furniture Factory Problem is formulated as a linear programming programming problem using the Gurobi Python API. The Furniture Factory Problem has been taken from the Saul Gass book: An illustrated Guide to Linear Programming.

To fully understand the content of this notebook, the reader should:

* Be familiar with Python.
* Have a background in any branch of engineering, computer science, economics, statistics, any branch of the “hard” sciences, or any discipline that uses quantitative models and methods.

The reader should also consult the  [documentation](https://www.gurobi.com/resources/?category-filter=documentation)
of the Gurobi Python API.

**Download the Repository** <br />
You can download the repository containing this and other examples by clicking [here](https://github.com/Gurobi/modeling-examples/archive/master.zip). 

**Gurobi License** <br />
In order to run this Jupyter Notebook properly, you must have a Gurobi license. If you do not have one, you can request an [evaluation license xxx](xxx) as a *commercial user*, or download a [free license xxx](xxx) as an *academic user*.

## The Furniture Factory Problem

A data scientist is in charge of developing the weekly production plan of two key products that the furniture factory makes: chairs and tables. The data scientist using machine learning techniques predicts that the selling price of a chair is $\$45$ and the selling price of a table is $\$80$ dollars. There are two critical resources in the production of chairs and tables:
Mahogany (measured in board square-feet) and labor (measured in work hours). There are 400 units of mahogany available at the beginning of each week, and there are 450 units of labor available during each week. The data scientist estimates that one chair requires 5 units of mahogany and 10 units of labor. One table requires 20 units of mahogany and 15 units of labor. The marketing department has told the data scientist that ALL the production of chairs and tables can be sold.

The goal is to create a production plan that maximizes total revenue.

To determine a production plan, we need to decide how many chairs and tables to make in order to maximize total revenue, while satisfying resources constraints. This problem has two decision variables:
* x1: number of chairs to produce.
* x2: number of tables to produce. 

The number of chairs and tables to produce should be a non-negative number. That is, x1, x2 ≥ 0. The data of the furniture problem can be summarized in the table below. 

![problem_summary1](problem_summary1.PNG)

We present now the Furniture Factory Problem in a parametrized formulation. The parametrized formulation separates the data of the problem from the model formulation.

In [1]:
# pip install for a limited license of the Gurobi callable library
# %pip install gurobipy

## Python Implementation

We import the Gurobi Python Module.

In [2]:
import gurobipy as gp
from gurobipy import GRB

# tested with Python 3.7.0 & Gurobi 9.1.0

## Input Data

We define all the input data for the model.

In [3]:
# resources data
resources, capacity = gp.multidict({
  'mahogany': 400,
  'labor':  450 })

The `gp.multidict()` function returns a list which maps each resource (key) to its capacity value.

In [4]:
# products data,
products, price = gp.multidict({
  'chair': 45,
  'table':  80 })

The `gp.multidict()` function returns a list which maps each resource (key) to its price value.

In [5]:
# Bill of materials: resources required by each product
bom = {
  ('mahogany', 'chair'):  5,
  ('mahogany', 'table'): 20,
  ('labor',    'chair'): 10,
  ('labor',    'table'): 15 }

The bom dictionary has a 2-tuple as a key, mapping the resource required by a product with its quantity per.

## Model Deployment

We now define the data structures of the parametrized model.

### Sets and Indices

$r \in \text{resources}$: Index and set of resources.

$p \in \text{products}$: Index and set of products.

### Parameters

$\text{price}_p \geq 0$: Price of product $p$.

$\text{capacity}_r \geq 0$: Capacity of resource $r$.

$\text{bom}_{r,p} \geq 0$: Amount of resource $r$ required by product $p$.

The `Model()` constructor creates a model object f . The name of this new model is again ‘Furniture’. This new model f initially contains no decision variables, constraints, or objective function.

In [6]:
# Create Furniture Factory  model
f = gp.Model("Furniture")

Using license file C:\Users\panit\gurobi.lic



### Decision Variables

$\text{make}_p \geq 0$: Number of products of type $p$ to build.

The `f.addVars()` method adds decision variables to the model object f, and returns a Gurobi tupledict object `make` that contains the variables recently created.
The first argument `products` provides the indices that will be used as keys to access the variables in the returned tupledict. The last argument gives the name ‘make’ to the  decision variables. The decision variables are of type continuous and non-negative, with no upper bound.

In [7]:
# Create decision variables for the products to make
make = f.addVars(products, name="make")

### Objective Function

The objective function is to maximize total revenue.

$$
\text{Max} \quad \text{revenue} = \sum_{p \: \in \: \text{products}} \text{price}_{p}*\text{make}_{p}
$$

The `f.setObjective()` method adds the objective function to the model object f.
The first argument is a linear expression which is generated by the `prod` method. The `prod` method is the product of the object `price` with the object `make` for each product p in the set `products`. The second argument defines the sense of the optimization.

In [8]:
# The objective is to maximize total revenue
f.setObjective(make.prod(price), GRB.MAXIMIZE)

## Constraints

The resources constraints establish that the consumpution of each raw material $r$ by building the products should not exceed the capacity available of the raw material.

$$
\sum_{p \: \in \: \text{products}} \text{bom}_{r,p}*\text{make}_{p} \leq \text{capacity}_r \quad \forall r \in \text{resources}
$$

The `addConstrs()` method adds constraints to the model object f, and returns a Gurobi tupledict object that contains the constraints recently created. We store the constrained generated in an object called `res`. The left-hand-side of each constraint can be created by using the `gp.quicksum()` method. The second argument is the sense of the constraint (≤). 
The third argument is the `capacity` value associated to the `resource`. 
Notice that one constraint will be generated per resource r in the set `resources`.
The last argument gives the name of the constraint.

In [9]:
# Create an object of type list to store the constraints for each resource

res = f.addConstrs(((gp.quicksum(bom[r,p]*make[p] for p in products) <= capacity[r]) for r in resources), name='R')

The `f.write()` method writes the LP problem formulation in the file 'furniture.lp'. This file is very helpful for debugging purposes. The furniture.lp file contains the LP formulation that Gurobi has in memory.

![furniture_lp](furniture_lp.PNG)

The `f.optimize()` method runs the optimization engine to solve the LP problem in the model object f

In [11]:
# Run optimization engine
f.optimize()

Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (win64)
Thread count: 4 physical cores, 8 logical processors, using up to 8 threads
Optimize a model with 2 rows, 2 columns and 4 nonzeros
Model fingerprint: 0xd0437183
Coefficient statistics:
  Matrix range     [5e+00, 2e+01]
  Objective range  [5e+01, 8e+01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [4e+02, 5e+02]
Presolve time: 0.00s
Presolved: 2 rows, 2 columns, 4 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    6.5000000e+31   2.968750e+30   6.500000e+01      0s
       2    2.2000000e+03   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds
Optimal objective  2.200000000e+03


In [12]:
# Display optimal production plan
for v in f.getVars():
    print(v.varName, v.x)

print(f"Optimal total revenue: ${f.objVal:,}")

make[chair] 24.0
make[table] 14.0
Optimal total revenue: $2,200.0


The `f.getVars()` method retrieves a list of all variables in the model object f. The print function displays the decision variable names `v.varName` and solution value `v.x`.

## Exercise 1

The goal of this exercise is for you to parametrized the diet problem as a linear programming model using the Gurobi Python API and find the optimal diet. This problem is defined in the `furniture_problem v1` Jupyter Notebook.

## Economic interpretation in Linear Programming models
+ Solving LP problems provides more information than only the values of the decision variables.
+ Associated with an LP optimal solution there are shadow prices for the constraints.
+ The shadow price of a constraint represents the change in the value of the objective function per unit of increase in the right-hand side value of that constraint.
+ There are shadow prices associated with the non-negativity constraints. These shadow prices are called the reduced costs.

In [13]:
# display shadow prices of resources constraints
for r in res.values():
    print(r.ConstrName, r.Pi)

R[mahogany] 1.0
R[labor] 4.0


## Exercise 2

Print the shadow prices of the diet problem constraints. Provide the economic interpretation for all the shadow prices. 

## References

Saul I. Gass, An illustrated guide to linear programming, Dover Publications; Revised ed. edition (March 1, 1990).

Copyright © 2021 Gurobi Optimization, LLC